In [1]:
import pandas as pd

TXT_AVALIABLE = True

In [2]:
if TXT_AVALIABLE:
    # Read the files into DataFrames
    avg_states = pd.read_fwf('avg_states.txt', dtype=str, header=None)
    min_states = pd.read_fwf('min_states.txt', dtype=str, header=None)
    max_states = pd.read_fwf('max_states.txt', dtype=str, header=None)

    print(avg_states.head())
    print(avg_states.tail())

    map = {
        0: "code",
        1: "jan",
        2: "feb",
        3: "mar",
        4: "apr",
        5: "may",
        6: "jun",
        7: "jul",
        8: "aug",
        9: "sep",
        10: "oct",
        11: "nov",
        12: "dec"
    }

    # Display the DataFrames
    avg_states = avg_states.rename(columns=map)
    min_states = min_states.rename(columns=map)
    max_states = max_states.rename(columns=map)

    avg_states.to_csv('avg_states.csv', index=False)
    min_states.to_csv('min_states.csv', index=False)
    max_states.to_csv('max_states.csv', index=False)

           0      1      2      3      4      5      6      7      8      9   \
0  0010021895  43.10  37.40  54.50  63.40  69.50  77.50  79.20  79.50  77.80   
1  0010021896  43.50  47.70  52.50  68.00  75.90  77.40  81.20  82.20  75.90   
2  0010021897  41.80  51.10  60.20  62.40  69.00  81.20  81.50  78.80  75.60   
3  0010021898  49.00  46.10  59.20  58.80  74.10  80.40  80.00  78.80  75.20   
4  0010021899  43.80  40.00  55.60  61.70  76.10  79.90  80.40  80.90  72.80   

      10     11     12  
0  59.70  53.20  44.90  
1  63.20  57.30  46.40  
2  67.10  54.20  47.40  
3  61.00  49.80  43.40  
4  66.00  55.30  44.70  
               0      1      2      3      4      5      6      7      8   \
12609  3650022020  47.60  48.10  58.30  61.00  69.90  77.60  82.40  81.10   
12610  3650022021  44.80  42.60  56.00  60.80  68.80  78.20  80.30  80.40   
12611  3650022022  43.00  45.80  55.00  62.50  72.40  79.60  83.30  80.40   
12612  3650022023  48.10  50.30  54.80  61.60  69.90  76.50  

In [3]:
# Parse the 'code' column to create 'country_code' and 'year' columns
min_states['year'] = min_states['code'].str[-4:]
min_states['code'] = min_states['code'].str[:3]

# Reorder columns to put 'year' as the second column
cols = min_states.columns.tolist()
cols.insert(1, cols.pop(cols.index('year')))
min_states = min_states[cols]

min_states = min_states[(min_states['code'].astype(int) <= 50) | (
    min_states['code'].astype(int) == 110)]  # keep only the states and "National"

In [4]:
# Parse the 'code' column to create 'country_code' and 'year' columns
max_states['year'] = max_states['code'].str[-4:]
max_states['code'] = max_states['code'].str[:3]

# Reorder columns to put 'year' as the second column
cols = max_states.columns.tolist()
cols.insert(1, cols.pop(cols.index('year')))
max_states = max_states[cols]

max_states = max_states[(max_states['code'].astype(int) <= 50) | (
    max_states['code'].astype(int) == 110)]

In [5]:
# Parse the 'code' column to create 'country_code' and 'year' columns
avg_states['year'] = avg_states['code'].str[-4:]
avg_states['code'] = avg_states['code'].str[:3]

# Reorder columns to put 'year' as the second column
cols = avg_states.columns.tolist()
cols.insert(1, cols.pop(cols.index('year')))
avg_states = avg_states[cols]

avg_states = avg_states[(avg_states['code'].astype(int) <= 50) | (
    avg_states['code'].astype(int) == 110)]

# Merge with state codes

In [6]:
# Read the state codes into a DataFrame
state_codes = pd.read_csv('state_codes.csv', dtype=str)

# Merge the state codes with avg_states, min_states, and max_states
avg_states = avg_states.merge(state_codes, on='code', how='left')
min_states = min_states.merge(state_codes, on='code', how='left')
max_states = max_states.merge(state_codes, on='code', how='left')

# Reorder columns
cols = avg_states.columns.tolist()
cols.insert(1, cols.pop(cols.index('country')))
avg_states = avg_states[cols]
min_states = min_states[cols]
max_states = max_states[cols]


avg_states.head()

,code,country,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,001,Alabama,1895,43.10,37.40,54.50,63.40,69.50,77.50,79.20,79.50,77.80,59.70,53.20,44.90
1,001,Alabama,1896,43.50,47.70,52.50,68.00,75.90,77.40,81.20,82.20,75.90,63.20,57.30,46.40
2,001,Alabama,1897,41.80,51.10,60.20,62.40,69.00,81.20,81.50,78.80,75.60,67.10,54.20,47.40
3,001,Alabama,1898,49.00,46.10,59.20,58.80,74.10,80.40,80.00,78.80,75.20,61.00,49.80,43.40
4,001,Alabama,1899,43.80,40.00,55.60,61.70,76.10,79.90,80.40,80.90,72.80,66.00,55.30,44.70


## Final dataset

We want to go from 

|code|country|year|jan|feb|mar|apr|may|jun|...
|-|-|-|-|-|-|-|-|-|-|

To

|code|country|year|month|min|max|avg|
|-|-|-|-|-|-|-|

In [7]:
min_melt = pd.melt(min_states, id_vars=[
                   "code", "country", "year"], var_name="month", value_name="min")
max_melt = pd.melt(max_states, id_vars=[
                   "code", "country", "year"], var_name="month", value_name="max")
avg_melt = pd.melt(avg_states, id_vars=[
                   "code", "country", "year"], var_name="month", value_name="avg")

In [8]:
print(f"min_states before melt:")
min_states.head()

min_states before melt:


,code,country,year,jan,feb,mar,apr,may,jun,jul,aug,sep,oct,nov,dec
0,001,Alabama,1895,33.40,26.80,42.40,51.20,58.40,66.50,68.80,69.30,66.40,45.20,41.40,32.70
1,001,Alabama,1896,34.00,36.40,41.20,55.60,63.90,66.90,70.70,70.40,62.60,50.60,45.80,35.10
2,001,Alabama,1897,31.40,41.20,50.40,50.10,55.90,68.30,70.20,68.10,62.50,53.10,41.40,36.90
3,001,Alabama,1898,38.80,34.20,48.50,46.40,60.20,68.00,69.60,69.20,65.10,49.60,38.70,32.50
4,001,Alabama,1899,33.10,28.60,43.50,50.80,63.60,67.60,68.90,69.90,59.40,54.00,42.80,33.80


In [9]:
print(f"min_states after melt:")
min_melt.head()

min_states after melt:


,code,country,year,month,min
0,001,Alabama,1895,jan,33.40
1,001,Alabama,1896,jan,34.00
2,001,Alabama,1897,jan,31.40
3,001,Alabama,1898,jan,38.80
4,001,Alabama,1899,jan,33.10


In [10]:
final_df = min_melt.merge(max_melt, on=["code", "country", "year", "month"]) \
    .merge(avg_melt, on=["code", "country", "year", "month"])

final_df.head()

,code,country,year,month,min,max,avg
0,001,Alabama,1895,jan,33.40,52.70,43.10
1,001,Alabama,1896,jan,34.00,53.00,43.50
2,001,Alabama,1897,jan,31.40,52.10,41.80
3,001,Alabama,1898,jan,38.80,59.10,49.00
4,001,Alabama,1899,jan,33.10,54.40,43.80


In [11]:
avg_states.to_csv('avg_states_final.csv', index=False)
min_states.to_csv('min_states_final.csv', index=False)
max_states.to_csv('max_states_final.csv', index=False)

In [12]:
final_df.to_csv("min_max_avg_states.csv", index=False)